In [174]:
import meanderpy as mp
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt

## Input parameters

In [175]:
W = 20.0                    # channel width (m)
D = 3.0                     # channel depth (m)
pad = 100                    # padding (number of nodepoints along centerline)
deltas = 50.0                # sampling distance along centerline
nit = 2000                   # number of iterations
Cf = 0.022                    # dimensionless Chezy friction factor
crdist = 1.5*W               # threshold distance at which cutoffs occur
kl = 60.0/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-11               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 20                # which time steps will be saved
n_bends = 30                 # approximate number of bends you want to model
Sl = 0.0                     # initial slope (matters more for submarine channels than rivers)
t1 = 500                    # time step when incision starts
t2 = 700                    # time step when lateral migration starts
t3 = 1400                    # time step when aggradation starts
aggr_factor = 4e-9           # aggradation factor (m/s, about 0.18 m/year, it kicks in after t3)

## Initialize model

In [179]:
from imp import reload
reload(mp)
ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
chb = mp.ChannelBelt(channels=[ch],cutoffs=[],cl_times=[0.0],cutoff_times=[]) # create channel belt object
diapir = mp.generate_diapir(radius=500, xmin=0, xmax=3000, ymin=-1000, ymax=1000) # create diapir

In [180]:
print('diapir center pt: ' + str(diapir[1]))
print('diapir radius: ' + str(diapir[0]))

diapir center pt: (1476, 785)
diapir radius: 500


## Run simulation

In [181]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,t1,t2,t3,aggr_factor,diapir) # channel migration
fig = chb.plot('morph',20,60,end_time=500, n_channels=100) # plotting

Percent: [####################] 99.95% 0000000001%   

## Build 3D fluvial model

In [7]:
h_mud = 0.4 # thickness of overbank deposit for each time step
dx = 10.0 # gridcell size in meters

chb_3d, xmin, xmax, ymin, ymax = chb.build_3d_model('fluvial',h_mud=h_mud,levee_width=4000.0,h=12.0,w=W,bth=0.0,
                            dcr=10.0,dx=dx,delta_s=deltas,starttime=chb.cl_times[20],endtime=chb.cl_times[-1],
                            xmin=0,xmax=0,ymin=0,ymax=0)

Percent: [####################] 98.76543209876543%  

In [8]:
# create plots
fig1,fig2,fig3 = chb_3d.plot_xsection(200, [[0.5,0.25,0],[0.9,0.9,0],[0.5,0.25,0]], 4)

## Build 3D submarine channel model

In [9]:
W = 200.0                    # channel width (m)
D = 12.0                     # channel depth (m)
pad = 50                    # padding (number of nodepoints along centerline)
deltas = 100.0                # sampling distance along centerline
nit = 1500                   # number of iterations
Cf = 0.02                    # dimensionless Chezy friction factor
crdist = 1.5*W               # threshold distance at which cutoffs occur
kl = 60.0/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-11               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 20                # which time steps will be saved
n_bends = 50                 # approximate number of bends you want to model
Sl = 0.01                     # initial slope (matters more for submarine channels than rivers)
t1 = 500                    # time step when incision starts
t2 = 700                    # time step when lateral migration starts
t3 = 1000                    # time step when aggradation starts
aggr_factor = 4.0          # aggradation factor (it kicks in after t3)

In [10]:
reload(mp)
ch = mp.generate_initial_channel(W,D,Sl,deltas,pad,n_bends) # initialize channel
chb = mp.ChannelBelt(channels=[ch],cutoffs=[],cl_times=[0.0],cutoff_times=[]) # create channel belt object

In [11]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,Cf,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration
fig = chb.plot('strat',20,60) # plotting

Percent: [####################] 99.93333333333332%   

In [12]:
h_mud = 3.0*np.ones((len(chb.cl_times[20:]),))
dx = 15.0

chb_3d, xmin, xmax, ymin, ymax = chb.build_3d_model('submarine',h_mud=h_mud,levee_width=5000.0,h=12.0,w=W,bth=6.0,
                            dcr=7.0,dx=dx,delta_s=deltas,starttime=chb.cl_times[20],endtime=chb.cl_times[-1],
                            xmin=0,xmax=0,ymin=0,ymax=0)


Percent: [####################] 98.21428571428571%  

In [15]:
fig1,fig2,fig3 = chb_3d.plot_xsection(100, [[0.5,0.25,0],[0.9,0.9,0],[0.5,0.25,0]], 10)